# Create a LangChain RAG system in Python with watsonx

**Contributors**: Nicholas Moss, Erika Russi

In this tutorial, we’ll use LangChain to walk through a step-by-step Retrieval Augmented Generation ([RAG](https://research.ibm.com/blog/retrieval-augmented-generation-RAG)) example in Python. 

For our use case, we’ll set up a RAG system for [IBM Think 2024](https://www.ibm.com/events/think). IBM Think 2024 is a conference where IBM announces new products, technologies, and partnerships. We will fetch content from several ibm.com websites, which will make up a knowledge base from, which we will provide to an LLM with context to answer some questions about Think 2024.

RAG is a technique in natural language processing (NLP) that combines information retrieval and generative models to produce more accurate, relevant and contextually aware responses. 

# More about RAG and LangChain

In traditional language generation tasks, [large language models](https://www.ibm.com/topics/large-language-models) (LLMs) like OpenAI’s GPT-3.5 (Generative Pre-trained Transformer) or [IBM’s Granite Models](https://www.ibm.com/granite) are used to construct responses based on an input prompt. However, these models may struggle to produce responses that are contextually relevant, factually accurate or up to date. The models may not know the latest information on IBM Think 2024. 

RAG applications address this limitation by incorporating a retrieval step before response generation. During retrieval, [vector search](https://www.ibm.com/topics/vector-search) can be used to identify contextually pertinent information, such as relevant information or documents from a large corpus of text, typically stored in a [vector database](https://www.ibm.com/topics/vector-database). Finally, an LLM is used to generate a response based on the retrieved context.

LangChain is a powerful, open-source framework that facilitates the development of applications using LLMs for various NLP tasks. In the context of RAG, LangChain plays a critical role by combining the strengths of retrieval-based methods and generative models to enhance the capabilities of NLP systems.

# Prerequisites

You need an [IBM Cloud account](https://cloud.ibm.com/registration?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-implement-xgboost-in-python&cm_sp=ibmdev-_-developer-_-trial) to create a [watsonx.ai](https://www.ibm.com/products/watsonx-ai?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-implement-xgboost-in-python&cm_sp=ibmdev-_-developer-_-product) project.

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this tutorial walks you through how to set up an IBM account to use a Jupyter Notebook. Jupyter Notebooks are widely used within [data science](https://www.ibm.com/topics/data-science) to combine code, text, images, and [data visualizations](https://www.ibm.com/topics/data-visualization) to develop a well-formed analysis.

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone) using your IBM Cloud account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas?topic=projects-creating-project#create-a-project).
   Take note of the project ID in project > Manage > General > Project ID. You'll need this ID for this tutorial.

3. Create a [Jupyter Notebook](https://www.ibm.com/docs/en/watsonx/saas?topic=editor-creating-managing-notebooks).

This step will open a Notebook environment where you can copy the code from this tutorial to implement a RAG application for Think 2024. Alternatively, you can download this notebook to your local system and upload it to your watsonx.ai project as an asset. To view more Granite tutorials, check out the [IBM Granite Community](https://github.com/ibm-granite-community). This tutorial is also available on [Github](https://github.com/IBM/ibmdotcom-tutorials/tree/main/generative-ai/langchain-rag.ipynb).

## Step 2. Set up watsonx.ai Runtime service and API key

1. Create a [watsonx.ai Runtime](https://cloud.ibm.com/catalog/services/watsonxai-runtime) service instance (choose the Lite plan, which is a free instance).

2. Generate an [API Key](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html). 

3. Associate the watsonx.ai Runtime service to the project you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=cpdaas). 

## Step 3. Install and import relevant libraries and set up credentials

We'll need a few libraries for this tutorial. Make sure to import the ones below, and if they're not installed, you can resolve this with a quick pip install.

In [ ]:
#installations
%pip install langchain
%pip install langchain_chroma
%pip install langchain-community
%pip install -U langchain_ibm
%pip install unstructured
%pip install ibm-watsonx-ai
%pip install nltk


Import the relevant libraries:

In [ ]:
#imports
import os
import nltk

from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv(os.getcwd()+"/.env", override=True)

nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
from dotenv import load_dotenv

load_dotenv(os.getcwd()+"/.env", override=True)


True

Set up your credentials. Store your `WATSONX_PROJECT_ID` and `WATSONX_APIKEY` in a separate `.env` file in the same level of your directory as this notebook.

In [ ]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("WATSONX_APIKEY", "")
}

Set up your `project_id` as part of your environment variables or input it:

In [ ]:
try:
    project_id = os.getenv("WATSONX_PROJECT_ID", "")
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

##  Step 4. Index the URLs to create the knowledge base

We’ll index our Think 2024 specific articles from URLs to create a knowledge base as a vectorstore. The content from these URLs will be our data sources and context for this exercise. The context will then be provided to an LLM to answer any questions we have about the Think 2024 conference.

The first step to building vector embeddings is to clean and process the raw dataset. This may involve the removal of noise and standardization of the text. For our example, we won’t do any cleaning since the text is already cleaned and standardized.

First, let's establish `URLS_DICTIONARY`. `URLS_DICTIONARY` is a dict that helps us map the URLs from which we will be extracting the content. Let's also set up a name for our collection: `askibm_think_2024`.

In [ ]:
URLS_DICTIONARY = {
    "ibm.com_events_think_faq.html": "https://www.ibm.com/events/think/faq",
    "events_think_agenda.html": "https://www.ibm.com/events/think/agenda",
    "products_watsonx_ai.html": "https://www.ibm.com/products/watsonx-ai",
    "products_watsonx_ai_foundation_models.html": "https://www.ibm.com/products/watsonx-ai/foundation-models",
    "watsonx_pricing.html": "https://www.ibm.com/watsonx/pricing",
    "watsonx.html": "https://www.ibm.com/watsonx",
    "products_watsonx_data.html": "https://www.ibm.com/products/watsonx-data",
    "products_watsonx_assistant.html": "https://www.ibm.com/products/watsonx-assistant",
    "products_watsonx_code_assistant.html": "https://www.ibm.com/products/watsonx-code-assistant",
    "products_watsonx_orchestrate.html": "https://www.ibm.com/products/watsonx-orchestrate",
    "products_watsonx_governance.html": "https://www.ibm.com/products/watsonx-governance",
    "granite_code_models_open_source.html": "https://research.ibm.com/blog/granite-code-models-open-source",
    "red_hat_enterprise_linux_ai.html": "https://www.redhat.com/en/about/press-releases/red-hat-delivers-accessible-open-source-generative-ai-innovation-red-hat-enterprise-linux-ai",
    "model_choice.html": "https://www.ibm.com/blog/announcement/enterprise-grade-model-choices/",
    "democratizing.html": "https://www.ibm.com/blog/announcement/democratizing-large-language-model-development-with-instructlab-support-in-watsonx-ai/",
    "ibm_consulting_expands_ai.html": "https://newsroom.ibm.com/Blog-IBM-Consulting-Expands-Capabilities-to-Help-Enterprises-Scale-AI",
    "ibm_data_product_hub.html": "https://www.ibm.com/products/data-product-hub",
    "ibm_price_performance_data.html": "https://www.ibm.com/blog/announcement/delivering-superior-price-performance-and-enhanced-data-management-for-ai-with-ibm-watsonx-data/",
    "ibm_bi_adoption.html": "https://www.ibm.com/blog/a-new-era-in-bi-overcoming-low-adoption-to-make-smart-decisions-accessible-for-all/",
    "watsonx_code_assistant_for_z.html": "https://www.ibm.com/blog/announcement/ibm-watsonx-code-assistant-for-z-accelerate-the-application-lifecycle-with-generative-ai-and-automation/",
    "code_assistant_for_java.html": "https://www.ibm.com/blog/announcement/watsonx-code-assistant-java/",
    "code_assistant_for_orchestrate.html": "https://www.ibm.com/blog/announcement/watsonx-orchestrate-ai-z-assistant/",
    "accelerating_gen_ai.html": "https://newsroom.ibm.com/Blog-How-IBM-Cloud-is-Accelerating-Business-Outcomes-with-Gen-AI",
    "watsonx_open_source.html": "https://newsroom.ibm.com/2024-05-21-IBM-Unveils-Next-Chapter-of-watsonx-with-Open-Source,-Product-Ecosystem-Innovations-to-Drive-Enterprise-AI-at-Scale",
    "ibm_concert.html": "https://www.ibm.com/products/concert",
    "ibm_consulting_advantage_news.html": "https://newsroom.ibm.com/2024-01-17-IBM-Introduces-IBM-Consulting-Advantage,-an-AI-Services-Platform-and-Library-of-Assistants-to-Empower-Consultants",
    "ibm_consulting_advantage_info.html": "https://www.ibm.com/consulting/info/ibm-consulting-advantage"
}
COLLECTION_NAME = "askibm_think_2024"

Next, let's load our documents using the LangChain `UnstructuredURLLoader` for the list of URLs we have. We'll print a sample document at the end to see how it's been loaded.

In [ ]:
documents = []

for url in list(URLS_DICTIONARY.values()):
    loader = UnstructuredURLLoader(urls=[url])
    data = loader.load()
    documents += data

#show sample document
documents[0]

#Output:

Based on the sample document, it looks like there's a lot of white space and new line characters that we can get rid of. Let's clean that up and add some metadata to our documents, including an id number and the source of the content.

In [ ]:
doc_id = 0
for doc in documents:
    doc.page_content = " ".join(doc.page_content.split()) # remove white space

    doc.metadata["id"] = doc_id #make a document id and add it to the document metadata

    print(doc.metadata)
    doc_id += 1

{'source': 'https://www.ibm.com/events/think/faq', 'id': 0}
{'source': 'https://www.ibm.com/events/think/agenda', 'id': 1}
{'source': 'https://www.ibm.com/products/watsonx-ai', 'id': 2}
{'source': 'https://www.ibm.com/products/watsonx-ai/foundation-models', 'id': 3}
{'source': 'https://www.ibm.com/watsonx/pricing', 'id': 4}
{'source': 'https://www.ibm.com/watsonx', 'id': 5}
{'source': 'https://www.ibm.com/products/watsonx-data', 'id': 6}
{'source': 'https://www.ibm.com/products/watsonx-assistant', 'id': 7}
{'source': 'https://www.ibm.com/products/watsonx-code-assistant', 'id': 8}
{'source': 'https://www.ibm.com/products/watsonx-orchestrate', 'id': 9}
{'source': 'https://www.ibm.com/products/watsonx-governance', 'id': 10}
{'source': 'https://research.ibm.com/blog/granite-code-models-open-source', 'id': 11}
{'source': 'https://www.redhat.com/en/about/press-releases/red-hat-delivers-accessible-open-source-generative-ai-innovation-red-hat-enterprise-linux-ai', 'id': 12}
{'source': 'https:/

Let's see how our sample document looks now after we cleaned it up:

In [ ]:
documents[0]

Document(metadata={'source': 'https://www.ibm.com/events/think/faq', 'id': 0}, page_content="View session catalog Event information Think 2025 will be held in Boston, MA at the Hynes Convention Center, 5-8 May. Think 2025 brings together senior business and technology leaders from across industries. Content is geared toward C-level, line of business and senior IT leaders. Think 2025 programming will be held at the Hynes Convention Center, centered in the heart of Boston’s most charming and historic Back Bay neighborhood. At IBM, we are committed to sustainability and environmentally responsible event planning. We are proud to partner with the Hynes Convention Center in Boston, where Green building practices include energy, water and waste conservation as part of their daily operating strategy to include minimizing power consumption through sensor-controlled utilities, and high-efficiency lighting. They are committed to using green cleaning products, sourcing local food and beverage, co

We need to split up our text into smaller, more manageable pieces known as "chunks". LangChain's `RecursiveCharacterTextSplitter` takes a large text and splits it based on a specified chunk size using a predefined set of characters. In order, the default characters are:
- "\n\n" - two new line characters 
- "\n" - one new line character
- " " - a space
- "" - an empty character

The process starts by attempting to split the text using the first character, "\n\n." If the resulting chunks are still too large, it moves to the next character, "\n," and tries splitting again. This continues with each character in the set until the chunks are smaller than the specified chunk size. Since we already removed all the "\n\n" and "\n" characters when we cleaned up the text, the `RecursiveCharacterTextSplitter` will begin at the " "(space) character.

We settled on a chunk size of 512 after experimenting with a chunk size of 1000. When the chunks were that large, our model was getting too much context for question-answering; this led to confused responses by the LLM because it was receiving too much information, so we changed it to smaller chunks. Feel free to experiment with chunk size further!


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Next, we choose an embedding model to be trained on our Think 2024 dataset. The trained embedding model is used to generate embeddings for each data point in the dataset. For text data, popular open-source embedding models include Word2Vec, GloVe, FastText or pre-trained transformer-based models like BERT or RoBERTa. OpenAIembeddings may also be used by leveraging the OpenAI embeddings API endpoint, the `langchain_openai` package and getting an `openai_api_key`, however, there is a cost associated with this usage.

Unfortunately, because the embedding models are so large, vector embedding often demands significant computational resources. We can greatly lower the costs linked to embedding vectors, while preserving performance and accuracy by using WatsonxEmbeddings. We'll use the IBM embeddings model, Slate, an encoder-only (RoBERTa-based) model, which while not generative, is fast and effective for many NLP tasks.



In [ ]:
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
    )

Let's load our content into a local instance of a vector database, using Chroma.

In [ ]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

Let's do a quick search of our vector database to test it out! Using `similarity_search_with_score` allows us to return the documents and the distance score of the query to them. The returned distance score is Euclidean distance. Therefore, a lower score is better.

In [ ]:
prompt = "What is IBM concert?"
search = vectorstore.similarity_search_with_score(prompt)
search

[(Document(metadata={'id': 23, 'source': 'https://newsroom.ibm.com/2024-05-21-IBM-Unveils-Next-Chapter-of-watsonx-with-Open-Source,-Product-Ecosystem-Innovations-to-Drive-Enterprise-AI-at-Scale'}, page_content='and solving problems before they happen. Concert will initially focus on helping application owners, SREs and IT leaders gain insights about, pre-empt and more quickly address issues around application risk and compliance management. Read this blog to learn more about IBM Concert. IBM expands ecosystem access to watsonx, adds third-party models IBM continues to foster a strong ecosystem of partners to offer clients choice and flexibility through bringing third-party models onto watsonx, enabling leading'),
  0.3013179302215576),
 (Document(metadata={'id': 23, 'source': 'https://newsroom.ibm.com/2024-05-21-IBM-Unveils-Next-Chapter-of-watsonx-with-Open-Source,-Product-Ecosystem-Innovations-to-Drive-Enterprise-AI-at-Scale'}, page_content='can easily move to and operate across multi

Let's ask "Where is Think 2024?"

In [ ]:
prompt = "Where is Think 2024?"
search = vectorstore.similarity_search_with_score(prompt)
search

[(Document(metadata={'id': 1, 'source': 'https://www.ibm.com/events/think/agenda'}, page_content='keynotes from Think 2024 and explore related content in the Think Hub. Watch on-demand Stay in the know. Catch up on all the action from Think 2024 and watch event replays. Watch on-demand'),
  0.20512937009334564),
 (Document(metadata={'id': 1, 'source': 'https://www.ibm.com/events/think/agenda'}, page_content='Think 2024 Think 2024 2024-05-20 Monday: IBM Partner Plus Day Programming for Business Partners 2024-05-21 Tuesday: Think programming kicks off Event registration begins. Scaling your business with AI and hybrid cloud. Join IBM and other industry experts for engaging sessions and demos in the Meeting Center. Unleash the transformative power of AI and automation. Chef-selected lunch items and connections with your peers. Full lunch menu in the Think mobile app. Scale productivity with watsonx assistants.'),
  0.3277530372142792),
 (Document(metadata={'id': 0, 'source': 'https://www.

## Step 5. Set up a retriever

We'll set up our vector store as a retriever. The retrieved information from the vector store serves as additional context or knowledge that can be used by a generative model.

In [ ]:
retriever = vectorstore.as_retriever()

## Step 6. Generate a response with a Generative Model

Finally, we’ll generate a response. The generative model (like GPT-4 or IBM Granite) uses the retrieved information to produce a more accurate and contextually relevant response to our questions about Think 2024.

First, we'll establish which LLM we're going to use to generate the response. For this tutorial, we'll use IBM's Granite-3.0-8B-Instruct model.

In [ ]:
model_id = "ibm/granite-3-8b-instruct"

The model parameters available can be found [here](https://ibm.github.io/watson-machine-learning-sdk/model.html#enums). We experimented with various model parameters, including Temperature, Top P, and Top K. [Here](https://www.ibm.com/docs/en/watsonx/saas?topic=lab-model-parameters-prompting)'s some more information on model parameters and what they mean.

In [ ]:
parameters = {
    GenParams.DECODING_METHOD: 'greedy',
    GenParams.TEMPERATURE: 2,
    GenParams.TOP_P: 0,
    GenParams.TOP_K: 100,
    GenParams.MIN_NEW_TOKENS: 10,
    GenParams.MAX_NEW_TOKENS: 512,
    GenParams.REPETITION_PENALTY:1.2,
    GenParams.STOP_SEQUENCES:['\n'],
    GenParams.RETURN_OPTIONS: {'input_tokens': True,'generated_tokens': True, 'token_logprobs': True, 'token_ranks': True, }
}

Next, we instantiate the LLM.

In [ ]:
llm = WatsonxLLM(
    model_id=model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

We'll set up a `prompttemplate` to ask multiple questions. The "context" will be derived from our retriever (our vector database) with the relevant documents and the "question" will be derived from the user query.

In [ ]:
template = """Generate a summary of the context that answers the question. Explain the answer in multiple steps if possible. 
Answer style should match the context. Ideal Answer Length 2-3 sentences.\n\n{context}\nQuestion: {question}\nAnswer:
"""
prompt = ChatPromptTemplate.from_template(template)

Let's set up a helper function to format the docs accordingly:

In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

And now we can set up a chain with our context, our prompt and our LLM. The generative model processes the augmented context along with the user's question to produce a response.

In [ ]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

And now we can ask multiple questions:

In [ ]:
chain.invoke("Where is Think 2024?")

"\nThe keynote events for Think 2024 were held in New York City, USA. However, it's important to note that Think 2025 has been announced as taking place in Boston, Massachusetts, USA."

Let's ask about IBM Concert next.

In [ ]:
chain.invoke("What is IBM Concert?")

"\nIBM Concert is a generative AI-powered tool set to launch in June 2024, serving as the 'nerve center' of an enterprise's technology and operations. It uses AI from watsonx to provide insights into clients' application portfolios, offering relevant operational and strategic recommendations to improve applications. Users can start their journey with a free trial or book a live demo."

And finally, let's ask what IBM Think 2024 is.

In [ ]:
chain.invoke("What is IBM Think 2024?")

'IBM Think 2024 is an upcoming event where IBM will showcase their latest advancements in technology, including AI and hybrid cloud solutions. The event will feature keynote speeches, engaging sessions, and networking opportunities for attendees. Additionally, IBM has announced plans to train over one million learners in AI globally by the end of 2026 through various initiatives such as SkillsBuild and AI Academy.'

And that's it! Feel free to ask even more questions!

# Summary and next steps

In this tutorial, you created a LangChain RAG system in Python with watsonx. You fetched 27 articles from https://www.ibm.com to create a vector store as context for an LLM to answer questions about the Think 2024 conference.

You can imagine a situation where we can create chatbots to field these questions.

We encourage you to check out the [LangChain documentation page](https://python.langchain.com/v0.2/docs/tutorials/rag/) for more information and tutorials on RAG.


## Try watsonx for free

Build an AI strategy for your business on one collaborative AI and data platform called [IBM watsonx](https://www.ibm.com/watsonx), which brings together new generative AI capabilities, powered by foundation models, and traditional machine learning into a powerful platform spanning the AI lifecycle. With [watsonx.ai](https://www.ibm.com/products/watsonx-ai), you can train, validate, tune, and deploy models with ease and build AI applications in a fraction of the time with a fraction of the data.

Try [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone), the next-generation studio for AI builders.

## Next steps

Explore more [articles and tutorials about watsonx](https://developer.ibm.com/components/watsonx/?) on IBM Developer.